<a href="https://colab.research.google.com/github/assamidanov/Solar_panel/blob/master/Solar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# downloaded = drive.CreateFile({'id':'1FOljIM5kbmIQ_WTaPGucTnUyM0H0Om1p'}) # replace the id with id of file you want to access
# downloaded.GetContentFile('personal.json') 
# with open('personal.json') as f:    
#     dt = json.load(f)

In [0]:
# downloaded = drive.CreateFile({'id':'1BOV91N3ZGwu6KDye7X0L98NBT--0TO45'}) # replace the id with id of file you want to access
# downloaded.GetContentFile('solar.json') 
# with open('solar.json') as f:
#     solar = json.load(f)

In [0]:
# def clean_air(df):
#     r=[]
#     for i in df:
#         try:
#             s = i['stations'][0]['dataList']
#             r.append(i)
#         except:
#             continue
#     return r
# def airquality_convert(df):
#     df = clean_air(df)
#     df_pre = json_normalize(json.loads(json_normalize(df,'stations').to_json(orient = "records")))
#     meta_info = [i for i in df_pre.drop(['dataList', ], axis=1)]
#     df = json_normalize(json.loads(json_normalize(json.loads(df_pre.to_json(orient = "records")),
#                                                   record_path = ['dataList'], meta=meta_info).to_json(orient = "records")))
#     df = pd.pivot_table(df, values='value', columns='type', index=['timestamp', "id"])
#     df = pd.DataFrame(df.to_records())
#     return df

In [0]:
# def clean_solar(df):
#     r=[]
#     for i in df:
#         try:
#             s = i['stations']
#             r.append(i)
#         except:
#             continue
#     return r
# def solarpanel_convert(df):
#     df = clean_solar(df)
#     df_pre = json_normalize(json.loads(json_normalize(df,'stations') .to_json(orient = "records")))
#     meta_info = ['{}.{}'.format(i, 'solar') for i in df_pre]
#     df_pre.columns = meta_info
#     meta_info = [i for i in df_pre.drop(['inverterList.solar', ], axis=1)]
#     df = json_normalize(json.loads(json_normalize(json.loads(df_pre.to_json(orient = "records")),
#                                                   record_path = ['inverterList.solar'], meta=meta_info).to_json(orient = "records")))
#     return df

In [0]:
# solar_df = solarpanel_convert(solar)

In [0]:
# air_quality = airquality_convert(dt)

In [0]:
# from google.colab import drive
# drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
# air_quality.to_csv('air_quality.csv', index = False)
# !cp air_quality.csv "drive/My Drive/ML Projects/Solar_Panel"

In [0]:
# solar_df.to_csv('solar_df.csv', index = False)
# !cp solar_df.csv "drive/My Drive/ML Projects/Solar_Panel"

In [0]:
downloaded = drive.CreateFile({'id':'1-2gt1Hd0GGDgGmFBV7NgDm3T0d_EHez9'}) # replace the id with id of file you want to access
downloaded.GetContentFile('solar_df.csv')
downloaded = drive.CreateFile({'id':'1m-T7PYEmC8nv1CNKFisNDg04YmASCjk-'}) # replace the id with id of file you want to access
downloaded.GetContentFile('air_quality.csv')
solar_df = pd.read_csv('solar_df.csv')
air = pd.read_csv('air_quality.csv')

In [0]:
df = pd.pivot_table(solar_df, values='powerGenerationParameters.powerGeneratedToday.solar',index=['timestamp.solar', "id.solar"], aggfunc = np.mean)
df = pd.DataFrame(df.to_records())

In [0]:
import time
df['timestamp.solar'] = df['timestamp.solar'].map(lambda x: time.strftime("%D %H:%M", time.localtime(int(x))))
df['timestamp.solar'] = pd.to_datetime(df['timestamp.solar'], errors='coerce')

In [0]:
def transform(df):
    df['hour'] = np.uint8(df['Date'].dt.hour)
    df['day'] = np.uint8(df['Date'].dt.day)
    df['weekday'] = np.uint8(df['Date'].dt.weekday)
    df['month'] = np.uint8(df['Date'].dt.month)
    df['year'] = np.uint8(df['Date'].dt.year-2000)
    return df

In [0]:
df.rename(columns = {'powerGenerationParameters.powerGeneratedToday.solar' : 'power',
                          'id.solar' : 'id',
                          'timestamp.solar': 'Date'},
               inplace = True)
df = transform(df)

In [0]:
df['date'] = df['Date'].dt.date
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [131]:
df_sample = pd.pivot_table(df, values='power',index=["day", "id", "month" ], aggfunc = ["count"])
df_sample = pd.DataFrame(df_sample.to_records())
df_jan = df_sample[df_sample['month']==1]
df_dec = df_sample[df_sample['month']==12]
print(df_jan.head())


print(df_dec.head())

   day     id  month  ('count', 'power')
0    1  43707      1                  26
1    1  55448      1                  34
2    1  58292      1                  32
3    1  58337      1                  25
4    1  59093      1                  23
     day                        id  month  ('count', 'power')
221   25                   ait82kw     12                   1
222   25                 aurobindo     12                   1
223   25              btcpusa110kw     12                   1
224   25  deendayalupadhyaycollege     12                   1
225   25                 delhihaat     12                   1


In [132]:
fig_1 = px.bar(df_jan, x='day', y="('count', 'power')", color='id')
fig_1.update_layout(title='Number of Observations from Each Stations in January')
fig_1.show()

In [133]:
fig_2 = px.bar(df_dec, x='day', y="('count', 'power')", color='id')
fig_2.update_layout(title='Number of Observations from Each Stations in December')
fig_2.show()

In [134]:
df['timestamp'] = df['date'].map(lambda x: x.strftime('%s'))
daily_df = df.copy()
daily_df = daily_df[['id', 'date','timestamp', 'power']]
daily_df_agg = daily_df.groupby( ['date', 'id','timestamp']).agg(['sum', 'mean', 'max', 'count'])
daily_df_agg = daily_df_agg.reset_index()
level_0 = daily_df_agg.columns.droplevel(0)
level_1 = daily_df_agg.columns.droplevel(1)
level_0 = ['' if x == '' else '-' + x for x in level_0]
daily_df_agg.columns = level_1 + level_0
daily_df_agg.rename_axis(None, axis=1)
less = daily_df_agg[daily_df_agg['power-count'] < 15] #threshhold
daily_df_agg = daily_df_agg[daily_df_agg['power-count'] > 15]
null = daily_df_agg[daily_df_agg['power-sum']==0]
daily_df_agg = daily_df_agg[daily_df_agg['power-sum']!=0]
daily_df_agg.head()

,date,id,timestamp,power-sum,power-mean,power-max,power-count
27,2019-12-30,ait82kw,1577664000,772.850012,24.151563,25.440001,32
28,2019-12-30,aurobindo,1577664000,525.529984,15.925151,17.219999,33
29,2019-12-30,btcpusa110kw,1577664000,1036.619967,30.488823,32.599998,34
30,2019-12-30,deendayalupadhyaycollege,1577664000,2000.300018,57.151429,61.110001,35
31,2019-12-30,delhihaat,1577664000,396.040003,11.315429,12.050000,35


In [135]:
fig_maximum = px.scatter(daily_df_agg, x='date', y='power-max', color='id', render_mode='svg')
fig_maximum.update_layout(title='Maximum kWh value per energy aspect')
fig_maximum.show()

In [0]:
for i, j in enumerate(less['id']):
  df= df[np.invert((df['timestamp']==less.iloc[i,2]) & (df['id']==j))]
for i, j in enumerate(null['id']):
  df= df[np.invert((df['timestamp']==null.iloc[i,2]) & (df['id']==j))]

In [137]:
fig_maximum = px.scatter(df, x='Date', y='power', color='id', render_mode='svg')
fig_maximum.update_layout(title='kWh value per energy aspect')
fig_maximum.show()